# GEOG0027 (2022/2023) Classification with Google Earth Engine (GEE)


This practical will use Google Earth Engine (GEE)'s python library [EE](https://developers.google.com/earth-engine) and [geemap library](https://geemap.org/) to rapidly visualise, analyse and download LandSat Images. These two libraries are built to handle remote sensing (RS) data from the Cloud without physically downloading the data to our local computers (unless we really want to), and they also allow easy python coding, where only small modifications are needed before handling large datasets. GEE hosts many popular RS datasets on the Cloud, and details of its data catalog can be found at: https://developers.google.com/earth-engine/datasets.

## Jupyter setup
1) If you are using UCL's [Jupyter Hub](https://jupyter.data-science.rc.ucl.ac.uk/) (VPN required) to work with this chapter, please run the following command line in a `Terminal` (found top right of the Jupyter_Hub page under `New`) first:*

`conda config --add envs_dirs /shared/groups/jrole001/geog0027/envs/`
![terminal-add-env](images/terminal_add_env.png)

2) To download this notebook, to execute and modify the code, run the following command in a `Terminal`:

`wget https://github.com/UCL-EO/GEOG0027/tree/main/docs/Intro_to_GEE.ipynb`

3) Then, run `source activate geemap` in terminal to activate the geemap environment;

4) Run `jupyter nbextension install --py --symlink --user ipyleaflet` to install Leaflet extension, and then run `jupyter nbextension enable ipyleaflet --user --py` to enable the extension;

5) Close `Terminal` and double check if the `jupyter-leaflet extension` is enabled (especially if no visible output map from running the examples below):

![leaflet](images/leaflet.png)

6) Run the follwing code cell. If you cannot see a Google Map showing on, then try to reboot Jupyter Hub to make jupyter-leaflet extension work. Before that, make sure you have saved all your current working notebooks. Then reboot Jupyter by clicking on Control Panel - Stop My Server - Start My Server.

![rebootJupyter](images/reboot.png)

## First map
To create a map we first need to define a location. Use google maps to pick a point and copy the coordinates.
![rebootJupyter](images/GEE_find_loc.png)

Enter the location in the cell below to define a map we use here. We start from displaying a basemap of the area. The first time when you use GEE, you will need to have a Google account and provide an authorization code as instructed.

In [1]:
import geemap, ee, os, numpy
import ipyleaflet

loc_coords = [51.49163903397572, -0.08313179193795397] ### EDIT THIS LINE WITH YOUR COORDS, [Lat,Lon] format

Map = geemap.Map(center=loc_coords, zoom=9)
Map

Map(center=[51.49163903397572, -0.08313179193795397], controls=(WidgetControl(options=['position', 'transparen…

A basic Google Map-like interface should be displayed here now. If you can't see anything, please ensure that the ipyleaflet nbextension has been enabled.

## Setting the Area Of Interest (AOI)
Now that we have a location to centre the Map on, we need to determine the size of the map. This can be done using a range in latitude/longitude using the function below. This cell just defines how a function work, and does not perform any actual calculations. To do this we then call the function.

In [2]:
def expand_coords(centre,lon_expand = 1.0,lat_expand = 0.5):
    lat1 = centre[0]-lat_expand
    lat2 = centre[0]+lat_expand
    lon1 = centre[1]-lon_expand
    lon2 = centre[1]+lon_expand
    return [lon1,lat1,lon2,lat2,]
    

This next piece of code will then calculate the map size. If the map isn't the correct size for your liking, add into the function alternative settings for 'lon_expand' and 'lat_expand' from those set in the previous cell.

In [3]:
rec_coords = expand_coords(loc_coords)
# rec_coords = expand_coords(loc_coords,lat_expand=0.7) ## example adjusted rectangle size
local_rec = ee.Geometry.Rectangle(rec_coords) 

## Examining the time series
Let's display a short 'movie' (a .gif file in fact) of how this area has changed over the past decades. This demonstrates the power of GEE - it is very quick to visualise a high volume of data.

In [4]:
Map_gif = geemap.Map(center=loc_coords, zoom=10)
Map_gif.add_landsat_ts_gif(roi=local_rec, start_year=1985, bands=['NIR', 'Red', 'Green'], frames_per_second=5)
Map_gif

Generating URL...
Please wait ...


KeyboardInterrupt: 

Next, we will compare such change by using a slider

In [17]:
landsat_ts = geemap.landsat_timeseries(roi=local_rec, start_year=1986, end_year=2020, \
                                       start_date='01-01', end_date='12-31')

layer_names = ['Landsat ' + str(year) for year in range(1986, 2022)]

geemap_landsat_vis = {
    'min': 0,
    'max': 3000,
    'gamma': [1, 1, 1],
    'bands': ['NIR', 'Red', 'Green']} # You can change the vis bands here

Map2 = geemap.Map()
Map2.ts_inspector(left_ts=landsat_ts, right_ts=landsat_ts, left_names=layer_names, right_names=layer_names, \
                 left_vis=geemap_landsat_vis, right_vis=geemap_landsat_vis)
Map2.centerObject(local_rec, zoom=10)
Map2

Map(center=[51.494062487824685, -0.08313179193795121], controls=(WidgetControl(options=['position', 'transpare…

We have previously defined a rectangle for Shenzhen area by coordinates, but we can also use existing shape/vector files to select Areas of Interest. 

## Load Landsat data collections from GEE
Now we can see the the buffered AOI displayed on the Map. Next, let's load some Landsat images for the elected area. I've defined here a python function called `display_landsat_collection` to do so. It automatically loads both the [surface reflectance](https://developers.google.com/earth-engine/datasets/catalog/LANDSAT_LC08_C01_T1_SR) and [annual NDVI](https://developers.google.com/earth-engine/datasets/catalog/LANDSAT_LC08_C01_T1_ANNUAL_NDVI) image collections from GEE's data catalog and also calculates the annual means for each band. 

You can skip most of the details of what's inside the code cell, but only to look at the first (and last) line of code. In order to run such function, you will need to choose a year (any year since 1984) and an AOI. In the following example, I choose year 2019 and the Shenzhen buffer to demonstrate the use of the code.

In [5]:
landsat_vis_param = {
            'min': 0,
            'max': 3000,
            'bands': ['NIR', 'Red', 'Green']  # False Colour Composit bands to be visualised 
}
ndvi_colorized_vis = {
            'min': 0.0,
            'max': 1.0,
            'palette': [
            'FFFFFF', 'CE7E45', 'DF923D', 'F1B555', 'FCD163', '99B718', '74A901',
            '66A000', '529400', '3E8601', '207401', '056201', '004C00', '023B01',
            '012E01', '011D01', '011301']
}

In [47]:
def load_landsat_collection(inmap,year, aoi, cloud_tolerance = 3.0, 
                            DISPLAY_ON_MAP = False, MEDIAN_ONLY = False):
    '''This function allows GEE to display a Landsat data collection 
    from any year between 1984 and present year
    that fall within the AOI and cloud tolerance, e.g. 3.0%.
    There are two optional flag:
    When DISPLAY_ON_MAP is TRUE, display this layer onto Map;
    When return_series = 'MEDIAN_ONLY', only median SR is loaded into landsat_ts, and
    Setting this option to MEDIAN_ONLY would be faster than loading other collections. 
    '''
    assert year >= 1984
    
    def renameBandsETM(image):
        if year >=2013: #LS8
            bands = ['B2', 'B3', 'B4', 'B5', 'B6', 'B7'] #, 'pixel_qa'
            new_bands = ['Blue', 'Green', 'Red', 'NIR', 'SWIR1', 'SWIR2'] #, 'pixel_qa'
        elif year <=1984:
            bands = ['B1', 'B2', 'B3', 'B4', 'B5', 'B7', 'pixel_qa']
            new_bands = ['Blue', 'Green', 'Red', 'NIR', 'SWIR1', 'SWIR2', 'pixel_qa']
        return image.select(bands).rename(new_bands)
        
    if not(MEDIAN_ONLY):
        if year >= 2013:
            layer_name = 'LC08' # LS8: 2013-now        
        elif year == 2012: # # LS7: 1999- , however SLC error >= 1999:
            layer_name = 'LE07' 
        elif year >=1984:
            layer_name = 'LT05' # LS5: 1984-2012
       
        collection_name_sr = f"LANDSAT/{layer_name}/C01/T1_SR" 
        # You can also use the following line, if interested in incorperating ndvi
        collection_name_ndvi = f"LANDSAT/{layer_name}/C01/T1_ANNUAL_NDVI" 

        all_sr_image = ee.ImageCollection(collection_name_sr) \
            .filterBounds(aoi) \
            .filterDate(f'{year}-01-01', f'{year}-12-31') \
            .filter(ee.Filter.lt('CLOUD_COVER', cloud_tolerance))\
            .sort('system:time_start') \
            .select('B[1-7]') \
            .sort('CLOUD_COVER')

        all_sr_image = all_sr_image.map(renameBandsETM) # rename bands with 'renameBandsETM' function
        
        # reduce all_sr_image to annual average per pixel
        mean_image = all_sr_image.mean()
        mean_image = mean_image.clip(aoi).unmask()

        ndvi_image = ee.ImageCollection(collection_name_ndvi)\
            .filterBounds(aoi) \
            .filterDate(f'{year}-01-01', f'{year}-12-31')\
            .select('NDVI')\
            .first()
        ndvi_image = ndvi_image.clip(aoi).unmask()

        #mean_image.addBands(ndvi_image, 'NDVI')
    
    # This line loads all annual median surface ref
    landsat_ts = geemap.landsat_timeseries(roi=aoi, start_year=year, end_year=year, \
                                       start_date='01-01', end_date='12-31')

    median_image = landsat_ts.first().clip(aoi).unmask()
    
    if DISPLAY_ON_MAP == True:
        
        if not(MEDIAN_ONLY):
            inmap.addLayer(ndvi_image, ndvi_colorized_vis, 'NDVI '+str(year),  opacity=0.9)
            inmap.addLayer(mean_image, landsat_vis_param, "Mean Ref "+str(year))
        inmap.addLayer(median_image, landsat_vis_param, "Median Ref "+str(year))

    if MEDIAN_ONLY:
        return median_image
    else:
        return all_sr_image, mean_image, median_image, ndvi_image 

Now the `load_landsat_collection` function has been defined, and we will run/execute it by calling the function name with appropriate input parameters (or 'arguments). The output of such function will be returned to the variables on the LHS of the equal sign, i.e. all_image_2019, mean_2019, median_2019, and ndvi_2019 in this case.

In [55]:
Map = geemap.Map(center=loc_coords, zoom=10)

# All you need to modify here is the YEAR below:
all_image_2019, mean_2019, median_2019, ndvi_2019 = load_landsat_collection(Map,2019,\
                                        local_rec, cloud_tolerance = 2,\
                                        DISPLAY_ON_MAP = True)
Map

Map(center=[51.49163903397572, -0.08313179193795397], controls=(WidgetControl(options=['position', 'transparen…

You should examine the functions under the `toolbar` and `layer` buttons on the top-right corner of the Map, e.g. use the `inspector` and `plotting` tools to check the data values, or use `layers` control to switch on/off layers or to adjust opacity.

We can also check the metadata from the Landsat image collection we just loaded from the Cloud. Have a look of the output. Any useful information?

In [64]:
first_image = all_image_2019.first() 

props = geemap.image_props(first_image)
print( props.getInfo())

{'CLOUD_COVER': 0.01, 'CLOUD_COVER_LAND': 0.03, 'EARTH_SUN_DISTANCE': 0.989582, 'ESPA_VERSION': '2_23_0_1b', 'GEOMETRIC_RMSE_MODEL': 8.908, 'GEOMETRIC_RMSE_MODEL_X': 6.443, 'GEOMETRIC_RMSE_MODEL_Y': 6.152, 'IMAGE_DATE': '2019-02-24', 'IMAGE_QUALITY_OLI': 9, 'IMAGE_QUALITY_TIRS': 9, 'LANDSAT_ID': 'LC08_L1TP_201025_20190224_20190308_01_T1', 'LEVEL1_PRODUCTION_DATE': 1552088520000, 'NOMINAL_SCALE': 30, 'PIXEL_QA_VERSION': 'generate_pixel_qa_1.6.0', 'SATELLITE': 'LANDSAT_8', 'SENSING_TIME': '2019-02-24T10:52:30.0138839Z', 'SOLAR_AZIMUTH_ANGLE': 157.231583, 'SOLAR_ZENITH_ANGLE': 62.337173, 'SR_APP_VERSION': 'LaSRC_1.3.0', 'WRS_PATH': 201, 'WRS_ROW': 25, 'system:asset_size': '392.213325 MB', 'system:band_names': ['Blue', 'Green', 'Red', 'NIR', 'SWIR1', 'SWIR2'], 'system:id': 'LANDSAT/LC08/C01/T1_SR/LC08_201025_20190224', 'system:index': 'LC08_201025_20190224', 'system:time_end': '2019-02-24 10:52:30', 'system:time_start': '2019-02-24 10:52:30', 'system:version': 1563396284578517}


The images shown so far are cropped agregates for the region we chose. To see the individual images involved in the composite, first get all the image names:

In [66]:
# gtest.getInfo()
image_list = all_image_2019.aggregate_array('system:id')
gtest.getInfo()

['LANDSAT/LC08/C01/T1_SR/LC08_201025_20190224',
 'LANDSAT/LC08/C01/T1_SR/LC08_201025_20190920',
 'LANDSAT/LC08/C01/T1_SR/LC08_202024_20190420',
 'LANDSAT/LC08/C01/T1_SR/LC08_202025_20190420',
 'LANDSAT/LC08/C01/T1_SR/LC08_202025_20190215',
 'LANDSAT/LC08/C01/T1_SR/LC08_203024_20190918']

Is this list longer or shorter than you expected? What happens when you adust `cloud_tolerance = ?` in the previous function call?

Copy one of the images from the list generated by the previous code cell into the cell below. 

We will start by getting the band names. 

In [57]:
image_name = 'EXAMPLE/IMAGE/NAME_DATE'
single_image = ee.Image(image_name)

single_image.bandNames().getInfo()

['B1',
 'B2',
 'B3',
 'B4',
 'B5',
 'B6',
 'B7',
 'B10',
 'B11',
 'sr_aerosol',
 'pixel_qa',
 'radsat_qa']

Previously the bands were renamed, but for a single image, we need to use the original band names. These are typically B[1-8], though you may need to adjust these depending on the year chosen.

In [120]:
landsat_vis = {'min': 0,'max': 3000,'bands': ['B4', 'B3', 'B2']}

Map.addLayer(single_image,landsat_vis,image_name)

Map

Map(bottom=87364.0, center=[51.57621608189103, -0.12222290039062501], controls=(WidgetControl(options=['positi…

Next, examine the mean, median surface reflectance and/or NDVI layers we've visualized. Which one is better? Which band should we include into the classification?


In addition to switching layers on and off, adjusting opacity, we can also use python code to some simple mathmatical operations, e.g. calculating the differences:

In [93]:
mean_ndvi = mean_2019.normalizedDifference(['NIR', 'Red'])
median_ndvi = median_2019.normalizedDifference(['NIR', 'Red'])
median_ndwi = median_2019.normalizedDifference(['Green','NIR'])

Map.addLayer(median_ndvi, ndvi_colorized_vis, 'Median NDVI')
Map.addLayer(mean_ndvi.subtract(median_ndvi), {'min': -0.2,'max': 0.2}, 'Diff in NDVI')
Map.addLayer(median_ndwi, ndvi_colorized_vis, 'NDWI from Median LS')
Map

Map(bottom=43977.0, center=[51.306579455859385, 0.22521972656250003], controls=(WidgetControl(options=['positi…

## Applying kernels to the data

In [107]:
high_pass=ee.Kernel.fixed(3,3,[[-1,-1,-1],[-1,8,-1],[-1,-1,-1]],1,1,False)
low_pass=ee.Kernel.fixed(3,3,[[1/9,1/9,1/9],[1/9,1/9,1/9],[1/9,1/9,1/9]],1,1,False)
# Map.addLayer(median_2019.convolve(high_pass),landsat_vis_param,'FCC high passed')
Map.addLayer(median_2019.convolve(low_pass),landsat_vis_param,'FCC low passed')

Map

Help on function Kernel.fixed in Kernel:

Kernel.fixed(*args, **kwargs)
    Creates a Kernel.
    
    Args:
      width: The width of the kernel in pixels.
      height: The height of the kernel in pixels.
      weights: A 2-D list of [height] x [width] values to use as the
          weights of the kernel.
      x: The location of the focus, as an offset from the left.
      y: The location of the focus, as an offset from the top.
      normalize: Normalize the kernel values to sum to 1.



Map(bottom=44057.0, center=[51.16901107945215, 0.23620605468750003], controls=(WidgetControl(options=['positio…

In [119]:

Map.addLayer(majority_ndvi_4,ndvi_colorized_vis,'3x3 majority analysed NDVI')

Map

Map(bottom=11309.0, center=[50.55532498251967, 0.9667968750000001], controls=(WidgetControl(options=['position…

## Mapping regions with high Vegetation or Water Index

In [123]:
ndvi_gt = median_ndvi.gt(median_ndvi.constant(0.5))
Map.addLayer(ndvi_gt,{},'NDVI > 0.5')
ndwi_gt=median_ndwi.gt(median_ndwi.constant(0.0))
Map.addLayer(ndwi_gt,{},'NDWI > 0.0')

In [126]:
majority_ndvi = ndvi_gt.reduceNeighborhood(
                reducer=ee.Reducer.mode(), 
                kernel=ee.Kernel.fixed(3,3,[[1]*3]*3,1,1,False) #3x3
#                 kernel=ee.Kernel.fixed(9,9,[[1]*9]*9,5,5,False) #9x9
#                 kernel=ee.Kernel.circle(4)
                )
Map.addLayer(majority_ndvi,{},'NDVI > 0.0, 3x3 majority')

## Download and save the images